 # 8 Modtran File Processing
 
This notebook forms part of a series on [computational optical radiometry](https://github.com/NelisW/ComputationalRadiometry#computational-optical-radiometry-with-pyradi).  The notebooks can be downloaded from [Github](https://github.com/NelisW/ComputationalRadiometry#computational-optical-radiometry-with-pyradi). These notebooks are constantly revised and updated, please revisit from time to time.  
 
 [<img src="https://zenodo.org/badge/doi/10.5281/zenodo.9910.png"   align="left"/>](http://dx.doi.org/10.5281/zenodo.9910)
 
 The date of this document and module versions used in this document are given at the end of the file.  
 Feedback is appreciated: neliswillers at gmail dot com.
 

## Overview

 The pyradi library has a module to handle Modtran files.  The module currently has only one function: to read Modtran tape7 files in an intelligent manner.  The function is [`rymodtran.loadtape7`](http://nelisw.github.io/pyradi-docs/_build/html/rymodtran.html#pyradi.rymodtran.loadtape7).


In [1]:
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import pyradi.ryfiles as ryfiles
import pyradi.ryplot as ryplot
import pyradi.rymodtran as rymodtran
import pyradi.ryutils as ryutils

from IPython.display import display
from IPython.display import Image
from IPython.display import HTML

import matplotlib as mpl
mpl.rc("savefig", dpi=300)
mpl.rc('figure', figsize=(10,8))
# %config InlineBackend.figure_format = 'svg'

# import pandas as pd
# pd.set_option('display.max_columns', 80)
# pd.set_option('display.width', 100)
# pd.set_option('display.max_colwidth', 150)



## Loading Tape7 Files

This function reads in the tape7 file from MODerate spectral resolution atmospheric TRANsmission ([MODTRAN](http://modtran5.com/)) code, that is used to model the propagation of the electromagnetic radiation through the atmosphere. tape7 is a primary file that contains all the spectral results of the MODTRAN run. The header information in the tape7 file contains portions of the tape5 information that is ignored here. The header section in tape7 is followed by a list of spectral samples with corresponding transmissions. Each column has a different component of the transmission or radiance. For more detail, see the Modtran documentation.




The user selects the appropriate columns by listing the column names, as
listed below. The tape7.scn file has missing columns, so this function does not work for tape7.scn files.  If you need a tape7.scn file with all the columns populated
you would have to use the regular tape7 file and convolve this to lower resolution.

The format of the tape7 file changes for different IEMSCT values. For
the most part the differences are hidden in the details, if the columns are read by column header and not by counting columns.  The various column headers used in the tape7 file are as follows:

**IEMSCT = 0** has two lines of column headers.  In order to select the column, you
must concatenate the two column headers with an underscore in between. All
columns are available with the following column names:   

`FREQ_CM-1`, `COMBIN_TRANS`, `H2O_TRANS`, `UMIX_TRANS`, `O3_TRANS`, `TRACE_TRANS`,
`N2_CONT`, `H2O_CONT`, `MOLEC_SCAT`, `AER+CLD_TRANS`, `HNO3_TRANS`,
`AER+CLD_abTRNS`, `-LOG_COMBIN`, `CO2_TRANS`, `CO_TRANS`, `CH4_TRANS`,
`N2O_TRANS`, `O2_TRANS`, `NH3_TRANS`, `NO_TRANS`, `NO2_TRANS`,
`SO2_TRANS`, `CLOUD_TRANS`, `CFC11_TRANS`, `CFC12_TRANS`, `CFC13_TRANS`,
`CFC14_TRANS`, `CFC22_TRANS`, `CFC113_TRANS`, `CFC114_TRANS`,
`CFC115_TRANS`, `CLONO2_TRANS`, `HNO4_TRANS`, `CHCL2F_TRANS`,
`CCL4_TRANS`,  `N2O5_TRANS`

**IEMSCT = 1** has single line of column headers. A number of columns have
headers, but with no column numeric data.  In the following list the
columns with header names `**` are empty in some runs:  

`FREQ`,`TOT_TRANS`, `PTH_THRML`, `THRML_SCT`, `SURF_EMIS`, `*SOL_SCAT*`,
`*SING_SCAT*`, `GRND_RFLT`, `*DRCT_RFLT*`, `TOTAL_RAD`, `*REF_SOL*`, `*SOL@OBS*`,
`DEPTH`, `DIR_EM`, `*TOA_SUN*`, `BBODY_T[K]`

Hence, these columns may not have valid data: 
`SOL_SCAT`, `SING_SCAT`, `DRCT_RFLT`, `REF_SOL`,`SOL@OBS`, `TOA_SUN`

**IEMSCT = 2** has a single line column headers. All the columns are available:  

`FREQ`, `TOT_TRANS`, `PTH_THRML`, `THRML_SCT`, `SURF_EMIS`, `SOL_SCAT`,
`SING_SCAT`, `GRND_RFLT`, `DRCT_RFLT`, `TOTAL_RAD`, `REF_SOL`, `SOL@OBS`,
`DEPTH`, `DIR_EM`, `TOA_SUN`, `BBODY_T[K]`

**IEMSCT = 3** has a single line column headers.  One of these seems to be two
words, which, in this code must be concatenated with an underscore. There
is also  additional column (assumed to be depth in this code).  The
columns available are  

`FREQ`, `TRANS`, `SOL_TR`, `SOLAR`, `DEPTH`

## Modtran Tape7 Reader Examples
First prepare by importing and downloading the tape7 files from the internet.


In [ ]:
get_ipython().system('dir')
tgzFilename = 'modtrandata.tgz'
destinationDir = '.'
tarFilename = 'modtrandata.tar'
url = 'https://raw.githubusercontent.com/NelisW/pyradi/master/pyradi/data/'
dlNames = ryfiles.downloadUntar(tgzFilename, url, destinationDir, tarFilename)

print('filesAvailable are {}'.format(dlNames))


The function [`rymodtran.loadtape7`](http://nelisw.github.io/pyradi-docs/_build/html/rymodtran.html#pyradi.rymodtran.loadtape7) is relatively easy to use, with only two parameters: the filename and the specification of which columns to read (a list). In some instances (`IEMSCT == 0`) the column header is given over two lines, and your specification must include the information on both lines. This is done by concatenating the two parts with an underscore character as in `TRACE` and `TRANS` becoming `TRACE_TRANS`.

 **IEMSCT = 0**: this example loads data from five columns. Note the underscore to indicate that the column header flows over two lines.


In [ ]:
tape7 = rymodtran.loadtape7("tape7-01", ['FREQ_CM-1', 'COMBIN_TRANS', 
                                         'MOLEC_SCAT', 'AER+CLD_TRANS', 
                                         'AER+CLD_abTRNS'] )
print(tape7.shape)


The next example reads all the columns in the tape7 file:

In [ ]:
tape7 = rymodtran.loadtape7("tape7-01", ['FREQ_CM-1', 'COMBIN_TRANS', 
                                         'H2O_TRANS', 'UMIX_TRANS', 
                                         'O3_TRANS', 'TRACE_TRANS', 
                                         'N2_CONT', 'H2O_CONT', 'MOLEC_SCAT',
                                         'AER+CLD_TRANS', 'HNO3_TRANS',
                                         'AER+CLD_abTRNS', '-LOG_COMBIN', 
                                         'CO2_TRANS', 'CO_TRANS', 'CH4_TRANS',
                                         'N2O_TRANS', 'O2_TRANS', 'NH3_TRANS',
                                         'NO_TRANS', 'NO2_TRANS', 'SO2_TRANS',
                                         'CLOUD_TRANS', 'CFC11_TRANS', 
                                         'CFC12_TRANS', 'CFC13_TRANS', 
                                         'CFC14_TRANS', 'CFC22_TRANS', 
                                         'CFC113_TRANS', 'CFC114_TRANS', 
                                         'CFC115_TRANS', 'CLONO2_TRANS', 
                                         'HNO4_TRANS', 'CHCL2F_TRANS', 
                                         'CCL4_TRANS', 'N2O5_TRANS'] )
print(tape7.shape)
